In [72]:
import pandas as pd
import numpy as np
import plotly as pt
import seaborn as sns
!pip install pymatgen
!pip install mp_api
import requests
import json
!pip install pymatgen nglview

In [73]:
from google.colab import output
output.enable_custom_widget_manager()

In [74]:
df = pd.read_excel("/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/Perovskite dataset export.xlsx",sheet_name='Photocatalytic dataset')

In [75]:
df.columns

Index(['Perovskite', 'Hill formula', 'Interlayer space composition',
       'Bandgap, eV', 'DOI', 'Materials Project ID', 'COD_ID', 'Springer_ID',
       'MP_CIF_modifier', 'COD_CIF_modifier', 'Springer_CIF_modifier',
       'Materials Project verification', 'COD verification',
       'Springer verification', 'General verification', 'MP_CIF_modified',
       'COD_CIF_modified', 'Springer_CIF_modified', 'Z', 'Z_MP', 'Z_COD',
       'Z_Springer', 'a, A', 'b, A', 'c, A', 'Symmetry group', 'd,A', 'a_MP',
       'b_MP', 'c_MP', 'a_COD', 'b_COD', 'c_COD', 'a_Springer', 'b_Springer',
       'c_Springer', 'Number of octahedrons on a layer', 'Valence electrons',
       'Volume', 'Volume_MP', 'Volume_COD', 'Volume_Springer',
       'Valence Electrons Density', 'Valence Electrons Density_MP',
       'Valence Electrons Density_COD', 'Springer_Valence Electrons Density',
       'avg s valence electrons', 'avg p valence electrons',
       'avg d valence electrons', 'avg f valence electrons',
       

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 88 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Perovskite                            1089 non-null   object 
 1   Hill formula                          730 non-null    object 
 2   Interlayer space composition          5 non-null      object 
 3   Bandgap, eV                           1054 non-null   float64
 4   DOI                                   1084 non-null   object 
 5   Materials Project ID                  1058 non-null   object 
 6   COD_ID                                935 non-null    object 
 7   Springer_ID                           290 non-null    object 
 8   MP_CIF_modifier                       499 non-null    object 
 9   COD_CIF_modifier                      197 non-null    object 
 10  Springer_CIF_modifier                 121 non-null    object 
 11  Materials Project

In [77]:
from pymatgen.core.structure import Structure
from pymatgen.core import Composition
from pymatgen.core.periodic_table import Element
import os
import re
import nglview as nv
from pymatgen.io.ase import AseAtomsAdaptor


In [78]:
subs_map = {
    "Ph": "C6H5",
    "Bn": "C7H7",
    #"Pr": "C3H7",
    "Bu": "C4H9",
    "Hx": "C6H13",
    "Me": "CH3",
    "Et": "C2H5",
    "Oc": "C8H17",
    "Dc": "C10H21",
}

import re

def expand_substituents(formula):
    if pd.isna(formula):
        return formula

    for abbr, full in subs_map.items():
        formula = re.sub(rf'{abbr}', full, formula)
    return formula

In [79]:
print(df.shape[0])
#df = df[~df['Perovskite'].str.contains("Nx", na=False)]
#df = df[~df['Perovskite'].str.contains("Ox", na=False)]
print(df.shape[0])
df['Perovskite'] = df['Perovskite'].apply(expand_substituents)

1089
1089


In [80]:
def getStructureFromCIF(cif_file_name):
  if(cif_file_name==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{cif_file_name}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',cif_file_name)
      return None
  else:
    return None

  if(structure == None):
    return None
  return structure

In [81]:
def eliminateAsterisksDromFormula(formula):
  parts = formula.split('*')
  main_formula = parts[0]
  if len(parts) == 1:
    return main_formula
  print("------------")
  print(formula)
  hydrate_part = parts[1]
  print(hydrate_part)
  #match = re.match(r'(\d+)([A-Za-z0-9]+)', hydrate_part)
  #match = re.match(r'([0-9]*\.?[0-9]+)([A-Za-z0-9]+)', hydrate_part)
  match = re.match(r'([0-9]*\.?[0-9]*)?([A-Za-z0-9]+)', hydrate_part)
  if not match:
     raise ValueError(f"Cannot parse hydrate: {hydrate_part}")
  number_str = match.group(1)
  n = float(number_str) if number_str else 1.0
  molecule = match.group(2)
  print(n)
  print(molecule)
  comp = Composition(molecule)
  comp *= n
  total_formula = Composition(main_formula) + comp
  print(total_formula)
  print("------------")
  return total_formula.reduced_composition

In [82]:
print(eliminateAsterisksDromFormula("CuSO4*5H2O"))
print(eliminateAsterisksDromFormula("CuSO4*0.25H2O"))

------------
CuSO4*5H2O
5H2O
5.0
H2O
Cu1 S1 O9 H10
------------
Cu1 S1 O9 H10
------------
CuSO4*0.25H2O
0.25H2O
0.25
H2O
Cu1 S1 O4.25 H0.5
------------
Cu1 S1 O4.25 H0.5


In [110]:
composition_relative_tolerance = 0.05

def checkCompositionStructureMatching(formula,cif_file_name):
  print('checkCompositionStructureMatching: entry')
  structure = getStructureFromCIF(cif_file_name)
  print('checkCompositionStructureMatching: structure is get')
  #print(structure)
  if(structure == None or structure==0):
    print('.cif file is not read')
    return False
  composition = structure.composition
  formula = eliminateAsterisksDromFormula(formula)
  try:
    composition_formula = Composition(formula)
  except:
    return False
  #print(type(composition))
  #print(type(composition_formula))
  #print(composition," || ", composition_formula, " = ")
  #print(composition_formula)
  #same = composition.reduced_composition == composition_formula.reduced_composition

  print(composition.items())
  factors = []
  for el, amt in composition.items():
    amt2= composition_formula[el];
    print(el," comp1: ",amt, " comp2: ",amt2)
    factor= amt2/amt
    factors.append(factor)

  factors_std = np.std(factors)
  print("Compositino scaling factors: ",factor," std:",factors_std)
  #for el, amt in comp.items():
  #      if el.symbol == from_el:
   #         for new_el, frac in to_dict.items():
  #              new_dict[Element(new_el)] = amt * frac

  #same = composition_formula.almost_equals(composition,rtol=0.4)
  same=False
  if(composition_relative_tolerance>factors_std):
    same=True

  print("CIF file:",cif_file_name," || " ,composition," || ", composition_formula, " = ",same)
  return same

In [111]:
#raise SystemExit()
#checkCompositionStructureMatching("Nb6K4O15OO","mp-560692")
#checkCompositionStructureMatching("HCa2Ta3O10*2C8H17NH2","HCa2Ta3O10_OcNH2")
checkCompositionStructureMatching("HCa2Nb3O10*2CH3NH2","HCa2Nb3O10_MeNH2")
#checkCompositionStructureMatching("K4La1.332Ta4O14","sd_1810747")

checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
.cif file is not read


False

#CIF modifier


In [85]:
def parse_stoichiometric_replacement(expr):
  expr = expr.replace(" ", "")
  if "->" not in expr:
        raise ValueError(f"Invalid expression (missing ->): {expr}")
  lhs, rhs = expr.split("->")
  print("LHS: ",lhs," RHS: ", rhs)

  # --- Parse LHS ---
  m = re.fullmatch(r"(?:(\d+(?:\.\d+)?))?([A-Z][a-z]?)", lhs)
  if not m:
      raise ValueError(f"Invalid LHS: {lhs}")

  lhs_coeff = float(m.group(1)) if m.group(1) else 1.0
  lhs_elem = m.group(2)

  # --- Parse RHS ---
  terms = rhs.split(",")
  rhs_counts = {}
  for term in terms:
        print('Term: ', term)
        #m = re.fullmatch(r"(\d+(?:\.\d+)?)([A-Z][a-z]?)", term)
        m = re.fullmatch(r"(?:(\d+(?:\.\d+)?))?([A-Z][a-z]?)", term)
        if not m:
            raise ValueError(f"Invalid RHS term: {term}")
        print("Term goups: ", m.group(1), "  ; ", m.group(2))
        coeff = 1
        if(m.group(1) is not None):
          coeff = float(m.group(1))
        elem = m.group(2)

        rhs_counts[elem] = rhs_counts.get(elem, 0.0) + coeff
  # --- Normalize RHS ---
  total_rhs = sum(rhs_counts.values())
  if total_rhs == 0:
      raise ValueError("RHS total stoichiometry is zero")

  rhs_fractions = {
      elem: coeff / total_rhs
      for elem, coeff in rhs_counts.items()
  }

  return {
      "from": lhs_elem,
      "total": lhs_coeff,
      "to": rhs_fractions
  }

In [86]:
inp = "Ta->Nb"
com = parse_stoichiometric_replacement(inp)
print(com)

inp = "Ta->0.5Nb,0.5Ta"
com = parse_stoichiometric_replacement(inp)
print(com)

LHS:  Ta  RHS:  Nb
Term:  Nb
Term goups:  None   ;  Nb
{'from': 'Ta', 'total': 1.0, 'to': {'Nb': 1.0}}
LHS:  Ta  RHS:  0.5Nb,0.5Ta
Term:  0.5Nb
Term goups:  0.5   ;  Nb
Term:  0.5Ta
Term goups:  0.5   ;  Ta
{'from': 'Ta', 'total': 1.0, 'to': {'Nb': 0.5, 'Ta': 0.5}}


In [87]:
def replace_element(comp, from_el, to_dict):
    print("Element replacement start: From ",from_el," To: ",to_dict)
    new_dict = {}

    for el, amt in comp.items():
        if el.symbol == from_el:
            for new_el, frac in to_dict.items():
                new_dict[Element(new_el)] = amt * frac
        else:
            new_dict[el] = amt
    output = Composition(new_dict)
    print("New comp: ", output)
    print("Element replacement is done!")
    return output

def modify_structure(structure, instruction):
  print("Start structure modification!")
  if(structure is None):
    print("Null structure")
    return None
  try:
    instructions = [cmd.strip() for cmd in instruction.split(";") if cmd.strip()]
        #old, new = instruction.split("->")
        #old = old.strip()
        #new = new.strip()
  except ValueError:
    raise ValueError("Failed to separte instructinos")
        #raise ValueError("Instruction must be of the form 'A->B', e.g. 'K->H'")
  output = structure
  for command in instructions:
    parsed_command = parse_stoichiometric_replacement(command)
    print("Parsed command: ", parsed_command)
    for site in structure:
      if site.is_ordered:
        print("Ordered site:", site.specie)
        if site.specie.symbol == parsed_command["from"]:
          site.species = {
              Element(el): frac
              for el, frac in parsed_command["to"].items()  #[TO DO]: not always 1:1 replacement
          }
      else:
        print("Disordered site:", site.species)
        print(site.species)
        print(type(site.species))
        species_comp = site.species;
        new_species_comp = replace_element(species_comp, parsed_command["from"], parsed_command["to"])
        site.species = new_species_comp
  print("Finish structure modification!")
  print("-------------------------------")
  print("-------------------------------")
  print("-------------------------------")
  print("-------------------------------")
  print("-------------------------------")
  return output


In [88]:
#s = getStructureFromCIF("sd_1810747")
#s
#s_new = modify_structure(s, "K->H")
s = getStructureFromCIF("sd_1958942")
s
s_new = modify_structure(s, "2Sr->Sr,Pb")
s_new.to("new_cif.cif","cif")

Start structure modification!
LHS:  2Sr  RHS:  Sr,Pb
Term:  Sr
Term goups:  None   ;  Sr
Term:  Pb
Term goups:  None   ;  Pb
Parsed command:  {'from': 'Sr', 'total': 2.0, 'to': {'Sr': 0.5, 'Pb': 0.5}}
Disordered site: Bi0.5 Sr0.5
Bi0.5 Sr0.5
<class 'pymatgen.core.composition.Composition'>
Element replacement start: From  Sr  To:  {'Sr': 0.5, 'Pb': 0.5}
New comp:  Bi0.5 Sr0.25 Pb0.25
Element replacement is done!
Disordered site: Bi0.5 Sr0.5
Bi0.5 Sr0.5
<class 'pymatgen.core.composition.Composition'>
Element replacement start: From  Sr  To:  {'Sr': 0.5, 'Pb': 0.5}
New comp:  Bi0.5 Sr0.25 Pb0.25
Element replacement is done!
Disordered site: Bi0.5 Sr0.5
Bi0.5 Sr0.5
<class 'pymatgen.core.composition.Composition'>
Element replacement start: From  Sr  To:  {'Sr': 0.5, 'Pb': 0.5}
New comp:  Bi0.5 Sr0.25 Pb0.25
Element replacement is done!
Disordered site: Bi0.5 Sr0.5
Bi0.5 Sr0.5
<class 'pymatgen.core.composition.Composition'>
Element replacement start: From  Sr  To:  {'Sr': 0.5, 'Pb': 0.5}
New

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: No structure parsed for section 1 in CIF.
'_atom_site_label'
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning: No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
  self.symmetry_operations = self.get_symops(data)  # type:ignore[assignment]
/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning: Cannot determine chemical composition from CIF! 'NoneType' object is not iterable
  if struct := self._get_structure(data, primitive, symmetrized, check_occu=check_occu):
/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning: No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
  self.symmetry_operations = self.get_symops(data)  # type:ignore[assignment]
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:310

"# generated using pymatgen\ndata_SrTa2Bi3PbClO11\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   3.91000000\n_cell_length_b   3.91000000\n_cell_length_c   18.49600000\n_cell_angle_alpha   90.00000000\n_cell_angle_beta   90.00000000\n_cell_angle_gamma   90.00000000\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   SrTa2Bi3PbClO11\n_chemical_formula_sum   'Sr1 Ta2 Bi3 Pb1 Cl1 O11'\n_cell_volume   282.76869760\n_cell_formula_units_Z   1\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Sr  Sr_fix1  1  0.50000000  0.50000000  0.73255000  0.25\n  Bi  Sr_fix1  1  0.50000000  0.50000000  0.73255000  0.5\n  Pb  Sr_fix1  1  0.50000000  0.50000000  0.73255000  0.25\n  Sr  Sr_fix1  1  0.50000000  0.50000000  0.26745000  0.25\n  Bi  Sr_fix1  1  0.50000000  0.50000000  

In [89]:
def modify_CIF(cif_file_name, instruction):
  print('modify CIF: entry', cif_file_name)
  structure = getStructureFromCIF(cif_file_name)
  print('modify CIF: structure is get')
  if(structure == None or structure==0):
    return None
  new_structure = modify_structure(structure, instruction)
  return new_structure

def modify_all_CIFs(cif_input_column, instruction_column, cif_output_column, prefix):
  results = []
  counter=0
  col_idx = {name: i for i, name in enumerate(df.columns)}
  cif_i = col_idx[cif_input_column]
  instr_i = col_idx[instruction_column]

  for i, row in enumerate(df.itertuples(index=False, name=None), start=1):
    cif_input = row[cif_i]
    instruction = row[instr_i]
    print("CIF input: ", cif_input, " instruction: ", instruction)
    if pd.isna(instruction):
      print("No instruction")
      results.append(cif_input)
      continue
    new_CIF = modify_CIF(cif_input,instruction)
    if(new_CIF is None):
      results.append(("Invalid structure to modify: "+cif_input))
      continue
    new_CIF_name = "M_"+ prefix +str(counter)
    counter = counter +1
    file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{new_CIF_name}.cif"
    new_CIF.to(file_path,"cif")
    results.append(new_CIF_name)

  df[cif_output_column] = results
  print("Modified CIFs: ", counter)


In [90]:
modify_all_CIFs("Materials Project ID", "MP_CIF_modifier", "MP_CIF_modified", "MP")

CIF input:  mp-560692  instruction:  nan
No instruction
CIF input:  mp-1223501  instruction:  nan
No instruction
CIF input:  mp-553965  instruction:  nan
No instruction
CIF input:  mp-553248  instruction:  nan
No instruction
CIF input:  mp-557195  instruction:  nan
No instruction
CIF input:  mp-20396  instruction:  nan
No instruction
CIF input:  mp-581330  instruction:  nan
No instruction
CIF input:  mp-557195  instruction:  Ca->Sr
modify CIF: entry mp-557195
modify CIF: structure is get
Start structure modification!
LHS:  Ca  RHS:  Sr
Term:  Sr
Term goups:  None   ;  Sr
Parsed command:  {'from': 'Ca', 'total': 1.0, 'to': {'Sr': 1.0}}
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Orde

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Nb0', 'Nb0', 'Nb1', 'Nb1', 'Bi2', 'Bi3', 'Pb4', 'O5', 'O6', 'O7', 'O8', 'O9', 'O10', 'O11', 'O12', 'O13']`.
  writer: Any = CifWriter(self, **kwargs)
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['K0', 'K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'K8', 'K9', 'K10', 'K11', 'K12', 'K13', 'K14', 'K15', 'Nb16', 'Nb16', 'Nb17', 'Nb17', 'Nb18', 'Nb18', 'Nb19', 'Nb19', 'Nb20', 'Nb20', 'Nb21', 'Nb21', 'Nb22', 'Nb22', 'Nb23', 'Nb23', 'Nb24', 'Nb24', 'Nb25', 'Nb25', 'Nb26', 'Nb26', 'Nb27', 'Nb27', 'Nb28', 'Nb28', 'Nb29', 'Nb29', 'Nb30', 'Nb30', 'Nb31', 'Nb31', 'Nb32', '

modify CIF: structure is get
Start structure modification!
LHS:  6Nb  RHS:  3Nb,3Ta
Term:  3Nb
Term goups:  3   ;  Nb
Term:  3Ta
Term goups:  3   ;  Ta
Parsed command:  {'from': 'Nb', 'total': 6.0, 'to': {'Nb': 0.5, 'Ta': 0.5}}
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Na0', 'Na0', 'Ca1', 'Ca2', 'Ta3', 'Ta4', 'Ta5', 'O6', 'O7', 'O8', 'O9', 'O10', 'O11', 'O12', 'O13', 'O14', 'O15']`.
  writer: Any = CifWriter(self, **kwargs)
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Na0', 'Na0', 'Na0', 'Ca1', 'Ca2', 'Ta3', 'Ta4', 'Ta5', 'O6', 'O7', 'O8', 'O9', 'O10', 'O11', 'O12', 'O13', 'O14', 'O15']`.
  writer: Any = CifWriter(self, **kwargs)
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifd

CIF input:  mp-1104930  instruction:  nan
No instruction
CIF input:  mp-1104930  instruction:  Ti->0.1Zr, 0.9Ti
modify CIF: entry mp-1104930
modify CIF: structure is get
Start structure modification!
LHS:  Ti  RHS:  0.1Zr,0.9Ti
Term:  0.1Zr
Term goups:  0.1   ;  Zr
Term:  0.9Ti
Term goups:  0.9   ;  Ti
Parsed command:  {'from': 'Ti', 'total': 1.0, 'to': {'Zr': 0.1, 'Ti': 0.9}}
Ordered site: K
Ordered site: K
Ordered site: La
Ordered site: La
Ordered site: Ti
Ordered site: Ti
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Finish structure modification!
-------------------------------
-------------------------------
-------------------------------
-------------------------------
-------------------------------
CIF input:  mp-1104930  instruction:  Ti->0.3Zr, 0.7Ti
modify CIF: entry mp-1104930
modify CIF: structure is get
Start structure modification!
LHS:  Ti  RHS:  0.3Zr,0.7Ti
Term:  0.3Zr
Term goups:  0.3 

...

Streaming output truncated to the last 5000 lines.
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Finish structure modification!
-------------------------------
-------------------------------
-------------------------------
-------------------------------
-------------------------------
CIF input:  K2La2Ti3O10_EtOH  instruction:  K->H; La->Nd
modify CIF: entry K2La2Ti3O10_EtOH
modify CIF: structure is get
Start structure modification!

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['K0', 'K1', 'La2', 'La2', 'La3', 'La3', 'Ti4', 'Ti5', 'Ti6', 'O7', 'O8', 'O9', 'O10', 'O11', 'O12', 'O13', 'O14', 'O15', 'O16']`.
  writer: Any = CifWriter(self, **kwargs)
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Na0', 'La1', 'La1', 'Ta2', 'Ta3', 'O4', 'O5', 'O6', 'O7', 'O8', 'O9', 'O10']`.
  writer: Any = CifWriter(self, **kwargs)


Streaming output truncated to the last 5000 lines.
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: H
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: C
Ordered site: N
Ordered site: N
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Finish structure modification!
-------------------------------
-------------------------------
-------------------------------
-------------------------------
-------------------------------
CIF input:  HCa2Ta3O10_MeNH2  instruction:  Ta->Nb
modify CIF: entry HCa2Ta3O10_MeNH2
modify CIF: stru

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Na0', 'Na1', 'La2', 'La3', 'Ti4', 'Ti4', 'Ti5', 'Ti5', 'Ti6', 'Ti6', 'O7', 'O8', 'O9', 'O10', 'O11', 'O12', 'O13', 'O14', 'O15', 'O16']`.
  writer: Any = CifWriter(self, **kwargs)
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Na0', 'Na0', 'Na1', 'Na1', 'La2', 'La3', 'Ti4', 'Ti5', 'Ti6', 'O7', 'O8', 'O9', 'O10', 'O11', 'O12', 'O13', 'O14', 'O15', 'O16']`.
  writer: Any = CifWriter(self, **kwargs)
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https:/

modify CIF: structure is get
Start structure modification!
LHS:  3Ti  RHS:  2Ti,Mn
Term:  2Ti
Term goups:  2   ;  Ti
Term:  Mn
Term goups:  None   ;  Mn
Parsed command:  {'from': 'Ti', 'total': 3.0, 'to': {'Ti': 0.6666666666666666, 'Mn': 0.3333333333333333}}
Ordered site: Na
Ordered site: Na
Ordered site: La
Ordered site: La
Ordered site: Ti
Ordered site: Ti
Ordered site: Ti
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Finish structure modification!
-------------------------------
-------------------------------
-------------------------------
-------------------------------
-------------------------------
CIF input:  mp-6144  instruction:  2Na->1.4H, 0.6Na
modify CIF: entry mp-6144
modify CIF: structure is get
Start structure modification!
LHS:  2Na  RHS:  1.4H,0.6Na
Term:  1.4H
Term goups:  1.4   ;  H
Term:  0.6Na
Term goups:  0.6   ;  Na
Parsed command:  {'from': 'Na', 

In [91]:
modify_all_CIFs("COD_ID", "COD_CIF_modifier", "COD_CIF_modified","COD")

CIF input:  1001842  instruction:  nan
No instruction
CIF input:  1545643  instruction:  nan
No instruction
CIF input:  -1  instruction:  nan
No instruction
CIF input:  2004917  instruction:  nan
No instruction
CIF input:  1521061  instruction:  nan
No instruction
CIF input:  2238958  instruction:  nan
No instruction
CIF input:  1518045  instruction:  nan
No instruction
CIF input:  -1  instruction:  nan
No instruction
CIF input:  -1  instruction:  nan
No instruction
CIF input:  1010942  instruction:  nan
No instruction
CIF input:  1001842  instruction:  nan
No instruction
CIF input:  1545643  instruction:  nan
No instruction
CIF input:  -1  instruction:  nan
No instruction
CIF input:  2004917  instruction:  nan
No instruction
CIF input:  1521061  instruction:  nan
No instruction
CIF input:  2238958  instruction:  nan
No instruction
CIF input:  1518045  instruction:  nan
No instruction
CIF input:  -1  instruction:  nan
No instruction
CIF input:  -1  instruction:  nan
No instruction
CIF 

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Cs1', 'Ca1', 'Ca1', 'Ta1', 'Ta2', 'Ta2', 'O3', 'O3', 'O3', 'O3', 'O2', 'O2', 'O4', 'O4', 'O1', 'O1']`.
  writer: Any = CifWriter(self, **kwargs)


modify CIF: structure is get
Start structure modification!
LHS:  Cs  RHS:  H
Term:  H
Term goups:  None   ;  H
Parsed command:  {'from': 'Cs', 'total': 1.0, 'to': {'H': 1.0}}
Ordered site: Cs
Ordered site: Ca
Ordered site: Ca
Ordered site: Ta
Ordered site: Ta
Ordered site: Ta
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
LHS:  Ta  RHS:  Nb
Term:  Nb
Term goups:  None   ;  Nb
Parsed command:  {'from': 'Ta', 'total': 1.0, 'to': {'Nb': 1.0}}
Ordered site: H
Ordered site: Ca
Ordered site: Ca
Ordered site: Ta
Ordered site: Ta
Ordered site: Ta
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Finish structure modification!
-------------------------------
-------------------------------
-------------------------------
-------------------------------
-----------------------

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Nb3', 'Nb3', 'Nb3', 'Nb3', 'Nb3', 'Nb3', 'Nb3', 'Nb3', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Bi2', 'Bi2', 'Bi1', 'Bi1', 'Bi3', 'Bi3', 'Bi3', 'Bi3', 'Pb2', 'Pb1', 'Pb3', 'Pb3', 'O9', 'O9', 'O9', 'O9', 'O9', 'O9', 'O9', 'O9', 'O8', 'O8', 'O8', 'O8', 'O8', 'O8', 'O8', 'O8', 'O2', 'O2', 'O4', 'O4', 'O1', 'O5', 'O5', 'O3', 'O6', 'O6', 'O6', 'O6', 'O7', 'O7', 'O7', 'O7', 'O7', 'O7', 'O7', 'O7']`.
  writer: Any = CifWriter(self, **kwargs)


CIF input:  1531664  instruction:  2Nb-> 1.9Nb, 0.1W
modify CIF: entry 1531664
modify CIF: structure is get
Start structure modification!
LHS:  2Nb  RHS:  1.9Nb,0.1W
Term:  1.9Nb
Term goups:  1.9   ;  Nb
Term:  0.1W
Term goups:  0.1   ;  W
Parsed command:  {'from': 'Nb', 'total': 2.0, 'to': {'Nb': 0.95, 'W': 0.05}}
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Bi
Ordered site: Bi
Ordered site: Bi
Ordered site: Bi
Ordered site: Bi
Ordered site: Bi
Ordered site: Bi
Ordered site: Bi
Ordered site: Pb
Ordered site: Pb
Ordered site: Pb
Ordered site: Pb
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered

...

modify CIF: structure is get
Start structure modification!
LHS:  K  RHS:  H
Term:  H
Term goups:  None   ;  H
Parsed command:  {'from': 'K', 'total': 1.0, 'to': {'H': 1.0}}
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: K
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Ca
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Na0', 'Na1', 'Na2', 'Na3', 'La4', 'La5', 'La6', 'La7', 'Ti1', 'Ti1', 'Ti1', 'Ti1', 'Ti2', 'Ti2', 'Ti2', 'Ti2', 'Ti2', 'Ti2', 'Ti2', 'Ti2', 'O1', 'O1', 'O1', 'O1', 'O2', 'O2', 'O2', 'O2', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3']`.
  writer: Any = CifWriter(self, **kwargs)


modify CIF: structure is get
Start structure modification!
LHS:  3Ti  RHS:  2.1Ti,0.9Mn
Term:  2.1Ti
Term goups:  2.1   ;  Ti
Term:  0.9Mn
Term goups:  0.9   ;  Mn
Parsed command:  {'from': 'Ti', 'total': 3.0, 'to': {'Ti': 0.7000000000000001, 'Mn': 0.3}}
Ordered site: Na
Ordered site: Na
Ordered site: Na
Ordered site: Na
Ordered site: La
Ordered site: La
Ordered site: La
Ordered site: La
Ordered site: Ti
Ordered site: Ti
Ordered site: Ti
Ordered site: Ti
Ordered site: Ti
Ordered site: Ti
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Finish structure modification!
-------------------------------
-------------------------------
-------------------------------
-------------------------------
----------------------------

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Na', 'Na', 'Na', 'Na', 'Na', 'Na', 'Na', 'Na', 'La8', 'La9', 'La10', 'La11', 'Ti1', 'Ti1', 'Ti1', 'Ti1', 'Ti2', 'Ti2', 'Ti2', 'Ti2', 'Ti2', 'Ti2', 'Ti2', 'Ti2', 'O1', 'O1', 'O1', 'O1', 'O2', 'O2', 'O2', 'O2', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3']`.
  writer: Any = CifWriter(self, **kwargs)


modify CIF: structure is get
Start structure modification!
LHS:  2Na  RHS:  1.4H,0.6Na
Term:  1.4H
Term goups:  1.4   ;  H
Term:  0.6Na
Term goups:  0.6   ;  Na
Parsed command:  {'from': 'Na', 'total': 2.0, 'to': {'H': 0.7, 'Na': 0.3}}
Ordered site: Na
Ordered site: Na
Ordered site: Na
Ordered site: Na
Ordered site: La
Ordered site: La
Ordered site: La
Ordered site: La
Ordered site: Ti
Ordered site: Ti
Ordered site: Ti
Ordered site: Ti
Ordered site: Ti
Ordered site: Ti
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
LHS:  3Ti  RHS:  2.995Ti,0.005Mn
Term:  2.995Ti
Term goups:  2.995   ;  Ti
Term:  0.005Mn
Term goups:  0.005   ;  Mn
Parsed command:  {'from': 'Ti', 'total': 3.0, 'to': {'Ti': 0.9983333333333334, 'Mn': 0.00

In [92]:
modify_all_CIFs("Springer_ID", "Springer_CIF_modifier", "Springer_CIF_modified","Springer")

CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction
CIF input:  nan  instruction:  nan
No instruction


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Rb1', 'Pr1', 'Ta1', 'Ta1', 'O1', 'O1', 'O1', 'O1', 'O2', 'O2', 'O3']`.
  writer: Any = CifWriter(self, **kwargs)


modify CIF: structure is get
Start structure modification!
LHS:  Pr  RHS:  La
Term:  La
Term goups:  None   ;  La
Parsed command:  {'from': 'Pr', 'total': 1.0, 'to': {'La': 1.0}}
Ordered site: Rb
Ordered site: Pr
Ordered site: Ta
Ordered site: Ta
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Finish structure modification!
-------------------------------
-------------------------------
-------------------------------
-------------------------------
-------------------------------
CIF input:  sd_1050391  instruction:  nan
No instruction
CIF input:  sd_1050391  instruction:  Pr->Nd
modify CIF: entry sd_1050391
modify CIF: structure is get
Start structure modification!
LHS:  Pr  RHS:  Nd
Term:  Nd
Term goups:  None   ;  Nd
Parsed command:  {'from': 'Pr', 'total': 1.0, 'to': {'Nd': 1.0}}
Ordered site: Rb
Ordered site: Pr
Ordered site: Ta
Ordered site: Ta
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered 

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Rb1', 'Rb1', 'Rb1', 'Rb1', 'Rb2', 'Rb2', 'Rb2', 'Rb2', 'Rb3', 'Rb3', 'Rb3', 'Rb3', 'Rb4', 'Rb4', 'Rb4', 'Rb4', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb1', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb2', 'Nb3', 'Nb3', 'Nb3', 'Nb3', 'Nb3', 'Nb3', 'Nb3', 'Nb3', 'Nb4', 'Nb4', 'Nb4', 'Nb4', 'Nb4', 'Nb4', 'Nb4', 'Nb4', 'O1', 'O1', 'O1', 'O1', 'O1', 'O1', 'O1', 'O1', 'O2', 'O2', 'O2', 'O2', 'O2', 'O2', 'O2', 'O2', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O3', 'O4', 'O4', 'O4', 'O4', 'O4', 'O4', 'O4', 'O4', 'O5', 'O5', 'O5', 'O5', 'O5', 'O5', 'O5', 'O5', 'O6', 'O6', 'O6', 'O6', 'O7', 'O7', 'O7', 'O7', 'O8', 'O8', 'O8', 'O8',

CIF input:  sd_0376709  instruction:  K->Rb
modify CIF: entry sd_0376709
modify CIF: structure is get
Start structure modification!
LHS:  K  RHS:  Rb
Term:  Rb
Term goups:  None   ;  Rb
Parsed command:  {'from': 'K', 'total': 1.0, 'to': {'Rb': 1.0}}
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Rb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: Nb
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordere

...

modify CIF: structure is get
Start structure modification!
LHS:  Li  RHS:  H
Term:  H
Term goups:  None   ;  H
Parsed command:  {'from': 'Li', 'total': 1.0, 'to': {'H': 1.0}}
Ordered site: Li
Ordered site: Li
Ordered site: Li
Ordered site: Li
Ordered site: Nd
Ordered site: Nd
Ordered site: Nd
Ordered site: Nd
Ordered site: Ta
Ordered site: Ta
Ordered site: Ta
Ordered site: Ta
Ordered site: Ta
Ordered site: Ta
Ordered site: Ta
Ordered site: Ta
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Ordered site: O
Finish structure modification!
-------------------------------
-------------------------------
----------

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Sr_fix1', 'Sr_fix1', 'Sr_fix1', 'Sr_fix1', 'Sr_fix3', 'Sr_fix3', 'Sr_fix3', 'Sr_fix3', 'Ta1', 'Ta1', 'Bi1', 'Cl1', 'O1', 'O1', 'O1', 'O1', 'O2', 'O2', 'O2', 'O2', 'O3', 'O3', 'O4']`.
  writer: Any = CifWriter(self, **kwargs)
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Ti_fix3', 'Ti_fix3', 'Ti_fix3', 'Ti_fix3', 'Sr_fix1', 'Sr_fix1', 'Sr_fix1', 'Sr_fix1', 'Sr_fix5', 'Sr_fix5', 'Sr_fix5', 'Sr_fix5', 'Bi1', 'Cl1', 'O1', 'O1', 'O1', 'O1', 'O2', 'O2', 'O2', 'O2', 'O3', 'O3', 'O4']`.
  writer: Any = CifWriter(self, **kwargs)


modify CIF: structure is get
Start structure modification!
LHS:  Nb  RHS:  Ta
Term:  Ta
Term goups:  None   ;  Ta
Parsed command:  {'from': 'Nb', 'total': 1.0, 'to': {'Ta': 1.0}}
Disordered site: Nb0.5 Ti0.5
Nb0.5 Ti0.5
<class 'pymatgen.core.composition.Composition'>
Element replacement start: From  Nb  To:  {'Ta': 1.0}
New comp:  Ta0.5 Ti0.5
Element replacement is done!
Disordered site: Nb0.5 Ti0.5
Nb0.5 Ti0.5
<class 'pymatgen.core.composition.Composition'>
Element replacement start: From  Nb  To:  {'Ta': 1.0}
New comp:  Ta0.5 Ti0.5
Element replacement is done!
Disordered site: Bi0.75 Sr0.25
Bi0.75 Sr0.25
<class 'pymatgen.core.composition.Composition'>
Element replacement start: From  Nb  To:  {'Ta': 1.0}
New comp:  Bi0.75 Sr0.25
Element replacement is done!
Disordered site: Bi0.75 Sr0.25
Bi0.75 Sr0.25
<class 'pymatgen.core.composition.Composition'>
Element replacement start: From  Nb  To:  {'Ta': 1.0}
New comp:  Bi0.75 Sr0.25
Element replacement is done!
Disordered site: Bi0.75 Sr0.2

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:286: UserWarning: Possible issue in CIF file at line: ?
  block = CifBlock.from_str(f"data_{block_str}")
/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1044: UserWarning: t parsed as T
  symbol = self._parse_symbol(label)
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: No structure parsed for section 3 in CIF.
could not convert string to float: 't'
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: No structure parsed for section 4 in CIF.
could not convert string to float: 't'
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetr

ERROR: Invalid structure for  sd_1210175
modify CIF: structure is get
CIF input:  sd_1210175  instruction:  2Na->1.4H, 0.6Na;3Ti->2.99Ti,0.01Mn
modify CIF: entry sd_1210175
ERROR: Invalid structure for  sd_1210175
modify CIF: structure is get
CIF input:  sd_1210175  instruction:  2Na->1.4H, 0.6Na;3Ti->2.975Ti,0.025Mn
modify CIF: entry sd_1210175
ERROR: Invalid structure for  sd_1210175
modify CIF: structure is get
CIF input:  sd_1210175  instruction:  2Na->1.4H, 0.6Na;3Ti->2.95Ti,0.05Mn
modify CIF: entry sd_1210175
ERROR: Invalid structure for  sd_1210175
modify CIF: structure is get
CIF input:  sd_1210175  instruction:  2Na->1.4H, 0.6Na;3Ti->2.9Ti,0.1Mn
modify CIF: entry sd_1210175
ERROR: Invalid structure for  sd_1210175
modify CIF: structure is get
CIF input:  sd_1210175  instruction:  2Na->1.6H, 0.4Na;3Ti->2.7Ti,0.3Mn
modify CIF: entry sd_1210175
ERROR: Invalid structure for  sd_1210175
modify CIF: structure is get
CIF input:  sd_1210175  instruction:  2Na->1.8H, 0.2Na;3Ti->2.5Ti,0

In [93]:
df.to_excel("checkpoint_CIF_modification.xlsx")

#CIF Verification

In [94]:
#def verifyCIFFilesColumn(column):
#df["Materials Project verification"] = df.apply(lambda row: checkCompositionStructureMatching(row['Perovskite'], row['Materials Project ID']), axis=1)
df["Materials Project verification"] = df.apply(lambda row: checkCompositionStructureMatching(row['Perovskite'], row['MP_CIF_modified']), axis=1)

checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element K, 16.0), (Element Nb, 24.0), (Element O, 68.0)])
K  comp1:  16.0  comp2:  4.0
Nb  comp1:  24.0  comp2:  6.0
O  comp1:  68.0  comp2:  17.0
Compositino scaling factors:  0.25  std: 0.0
CIF file: mp-560692  ||  K16 Nb24 O68  ||  K4 Nb6 O17  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element K, 1.0), (Element La, 1.0), (Element Nb, 2.0), (Element O, 7.0)])
K  comp1:  1.0  comp2:  1.0
La  comp1:  1.0  comp2:  1.0
Nb  comp1:  2.0  comp2:  2.0
O  comp1:  7.0  comp2:  7.0
Compositino scaling factors:  1.0  std: 0.0
CIF file: mp-1223501  ||  K1 La1 Nb2 O7  ||  K1 La1 Nb2 O7  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element Rb, 2.0), (Element La, 2.0), (Element Nb, 4.0), (Element O, 14.0)])
Rb  comp1:  2.0  comp2:  1.0
La  comp1:  2.0  comp2

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


Streaming output truncated to the last 5000 lines.
Nb  comp1:  10.800000000000002  comp2:  2.7
H  comp1:  4.0  comp2:  1.0
O  comp1:  40.0  comp2:  10.0
Compositino scaling factors:  0.25  std: 1.2412670766236365e-17
CIF file: M_MP211  ||  Ca8 Ta1.2 Nb10.8 H4 O40  ||  H1 Ca2 Nb2.7 Ta0.3 O10  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element Ca, 8.0), (Element Ta, 4.0), (Element Nb, 8.0), (Element H, 4.0), (Element O, 40.0)])
Ca  comp1:  8.0  comp2:  2.0
Ta  comp1:  4.0  comp2:  1.0
Nb  comp1:  8.0  comp2:  2.0
H  comp1:  4.0  comp2:  1.0
O  comp1:  40.0  comp2:  10.0
Compositino scaling factors:  0.25  std: 0.0
CIF file: M_MP212  ||  Ca8 Ta4 Nb8 H4 O40  ||  H1 Ca2 Nb2 Ta1 O10  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element Ca, 8.0), (Element Ta, 6.0), (Element Nb, 6.0), (Element H, 4.0), (Element O, 40.0)])
Ca  comp1:  8.0  comp2:  2.0
Ta  c

In [95]:
#df["COD verification"] = df.apply(lambda row: checkCompositionStructureMatching(row['Perovskite'], row['COD_ID']), axis=1)
df["COD verification"] = df.apply(lambda row: checkCompositionStructureMatching(row['Perovskite'], row['COD_CIF_modified']), axis=1)

checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Species K+, 16.0), (Species Nb5+, 24.0), (Species O2-, 68.0)])
K+  comp1:  16.0  comp2:  0
Nb5+  comp1:  24.0  comp2:  0
O2-  comp1:  68.0  comp2:  0
Compositino scaling factors:  0.0  std: 0.0
CIF file: 1001842  ||  K+16 Nb5+24 O2-68  ||  K4 Nb6 O17  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element K, 2.188), (Element La, 1.936), (Element Nb, 4.0), (Element O, 14.0)])
K  comp1:  2.188  comp2:  1.0
La  comp1:  1.936  comp2:  1.0
Nb  comp1:  4.0  comp2:  2.0
O  comp1:  14.0  comp2:  7.0
Compositino scaling factors:  0.5  std: 0.022046731764576926
CIF file: 1545643  ||  K2.188 La1.936 Nb4 O14  ||  K1 La1 Nb2 O7  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_item

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


checkCompositionStructureMatching: structure is get
dict_items([(Element La, 8.0), (Element Ba, 2.0), (Element Ti, 8.0), (Element O, 30.0)])
La  comp1:  8.0  comp2:  4.0
Ba  comp1:  2.0  comp2:  0
Ti  comp1:  8.0  comp2:  4.0
O  comp1:  30.0  comp2:  15.0
Compositino scaling factors:  0.5  std: 0.21650635094610965
CIF file: 1526777  ||  La8 Ba2 Ti8 O30  ||  Ca1 La4 Ti4 O15  =  False
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element Sr, 8.0), (Element Ta, 8.0), (Element O, 28.0)])
Sr  comp1:  8.0  comp2:  2.0
Ta  comp1:  8.0  comp2:  2.0
O  comp1:  28.0  comp2:  7.0
Compositino scaling factors:  0.25  std: 0.0
CIF file: 2106523  ||  Sr8 Ta8 O28  ||  Sr2 Ta2 O7  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Species Sr2+, 16.0), (Species Nb5+, 16.0), (Species O2-, 56.0)])
Sr2+  comp1:  16.0  comp2:  0
Nb5+  comp1:  16.0  comp2:  0
O2-  comp1:  56.0  comp2:  0

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning: Incorrect stoichiometry:
  CIF={'Ba': 1.0, 'Bi': 4.0, 'O': 15.0, 'Ti': 4.0}
  PMG={'Ti': 16.0, 'Ba': 3.76, 'Bi': 16.239999999999995, 'O': 60.0}
  ratios={'Ba': 3.76, 'Ti': 4.0, 'O': 4.0, 'Bi': 4.059999999999999}
  if struct := self._get_structure(data, primitive, symmetrized, check_occu=check_occu):


checkCompositionStructureMatching: structure is get
dict_items([(Element Na, 2.0), (Element La, 2.0), (Element Ta, 4.0), (Element O, 14.0)])
Na  comp1:  2.0  comp2:  1.0
La  comp1:  2.0  comp2:  1.0
Ta  comp1:  4.0  comp2:  2.0
O  comp1:  14.0  comp2:  7.0
Compositino scaling factors:  0.5  std: 0.0
CIF file: 1544432  ||  Na2 La2 Ta4 O14  ||  Na1 La1 Ta2 O7  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatchin

In [96]:
#df["Springer verification"] = df.apply(lambda row: checkCompositionStructureMatching(row['Perovskite'], row['Springer_ID']), axis=1)
df["Springer verification"] = df.apply(lambda row: checkCompositionStructureMatching(row['Perovskite'], row['Springer_CIF_modified']), axis=1)

checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatc

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:286: UserWarning: Possible issue in CIF file at line: No transformation from published to standardized cell parameters necessary.
  block = CifBlock.from_str(f"data_{block_str}")
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
t parsed as T
No structure parsed for section 3 in CIF.
could not convert string to float: 't'
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
t parsed as T
No structure parsed for section 4 in CIF.
could not convert string to float: 't'
  struct = parser.parse_structures(primitive=primitive)[0]


checkCompositionStructureMatching: structure is get
dict_items([(Element K, 1.0), (Element Nb, 1.0), (Element O, 3.0)])
K  comp1:  1.0  comp2:  1.0
Nb  comp1:  1.0  comp2:  1.0
O  comp1:  3.0  comp2:  3.0
Compositino scaling factors:  1.0  std: 0.0
CIF file: sd_0308503  ||  K1 Nb1 O3  ||  K1 Nb1 O3  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element Rb, 1.0), (Element La, 1.0), (Element Ta, 2.0), (Element O, 7.0)])
Rb  comp1:  1.0  comp2:  1.0
La  comp1:  1.0 

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: Pauling file corrections applied.
10 fractional coordinates rounded to ideal values to avoid issues with finite precision.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
  struct = parser.parse_structures(primitive=primitive)[0]


checkCompositionStructureMatching: structure is get
dict_items([(Element La, 4.0), (Element Sr, 1.0), (Element Ti, 4.0), (Element O, 15.0)])
La  comp1:  4.0  comp2:  4.0
Sr  comp1:  1.0  comp2:  1.0
Ti  comp1:  4.0  comp2:  4.0
O  comp1:  15.0  comp2:  15.0
Compositino scaling factors:  1.0  std: 0.0
CIF file: sd_0306445  ||  La4 Sr1 Ti4 O15  ||  Sr1 La4 Ti4 O15  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element La, 4.0), (Element Ca, 1.0), (Element Ti, 4.0), (Element O, 15.0)])
La  comp1:  4.0  comp2:  4.0
Ca  comp1:  1.0  comp2:  1.0
Ti  comp1:  4.0  comp2:  4.0
O  comp1:  15.0  comp2:  15.0
Compositino scaling factors:  1.0  std: 0.0
CIF file: sd_0306542  ||  La4 Ca1 Ti4 O15  ||  Ca1 La4 Ti4 O15  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructure

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1041: UserWarning: OH parsed as 
  symbol = self._parse_symbol(data["_atom_site_type_symbol"][idx])
/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
OH parsed as 
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
OH parsed as 
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
  struct = parser.parse_structures(primitive=primitive)[0]


checkCompositionStructureMatching: structure is get
dict_items([(Element Sr, 2.0), (Element Ta, 4.0), (Element O, 10.0)])
Sr  comp1:  2.0  comp2:  1.0
Ta  comp1:  4.0  comp2:  2.0
O  comp1:  10.0  comp2:  7.0
Compositino scaling factors:  0.7  std: 0.09428090415820632
CIF file: sd_1200159  ||  Sr2 Ta4 O10  ||  H2 Sr1 Ta2 O7  =  False
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element K, 4.002), (Element La, 1.33), (Element Ta, 4.0), (Element O, 14.0)])
K  comp1:  4.002  comp2:  6.0
La  comp1:  1.33  comp2:  2.0
Ta  comp1:  4.0  comp2:  6.0
O  comp1:  14.0  comp2:  21.0
Compositino scaling factors:  1.5  std: 0.0017628337366642317
CIF file: sd_1810747  ||  K4.002 La1.33 Ta4 O14  ||  K6 La2 Ta6 O21  =  True
checkCompositionStructureMatching: e

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
t parsed as T
No structure parsed for section 3 in CIF.
could not convert string to float: 't'
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.
t parsed as T
No structure parsed for section 4 in CIF.
could not convert string to float: 't'
  struct = parser.parse_structures(primitive=primitive)[0]


checkCompositionStructureMatching: structure is get
dict_items([(Element Na, 4.0), (Element Nd, 4.0), (Element Ta, 8.0), (Element O, 28.0)])
Na  comp1:  4.0  comp2:  1.0
Nd  comp1:  4.0  comp2:  1.0
Ta  comp1:  8.0  comp2:  2.0
O  comp1:  28.0  comp2:  7.0
Compositino scaling factors:  0.25  std: 0.0
CIF file: sd_1955782  ||  Na4 Nd4 Ta8 O28  ||  Na1 Nd1 Ta2 O7  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element K, 4.0), (Element Nd, 4.0), (Element Ta, 8.0), (Element O, 28.0)])
K  comp1:  4.0  comp2:  1.0
Nd  comp1:  4.0  comp2:  1.0
Ta  comp1:  8.0  comp2:  2.0
O  comp1:  28.0  comp2:  7.0
Compositino scaling factors:  0.25  std: 0.0
CIF file: sd_1538044  ||  K4 Nd4 Ta8 O28  ||  K1 Nd1 Ta2 O7  =  True
checkCompositionStructureMatching: entry
checkCompositionStructureMatching: structure is get
dict_items([(Element Rb, 1.0), (Element Nd, 1.0), (Element Ta, 2.0), (Element O, 7.0)])
Rb  comp1:  1.0  comp2:  1.0
Nd  com

In [97]:
def markEntriesWithoutVerifiedCIF(ver1, ver2, ver3):
  if(ver1 or ver2 or ver3):
    return False
  return True

In [98]:
df["General verification"] = df.apply(lambda row: markEntriesWithoutVerifiedCIF(row['Materials Project verification'], row['COD verification'],row['Springer verification']), axis=1)
df_filtered = df[df['General verification'] != True]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 985 entries, 0 to 1088
Data columns (total 88 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Perovskite                            985 non-null    object 
 1   Hill formula                          671 non-null    object 
 2   Interlayer space composition          4 non-null      object 
 3   Bandgap, eV                           954 non-null    float64
 4   DOI                                   980 non-null    object 
 5   Materials Project ID                  967 non-null    object 
 6   COD_ID                                860 non-null    object 
 7   Springer_ID                           283 non-null    object 
 8   MP_CIF_modifier                       472 non-null    object 
 9   COD_CIF_modifier                      197 non-null    object 
 10  Springer_CIF_modifier                 120 non-null    object 
 11  Materials Project verif

In [99]:
df_filtered.to_excel("checkpoint_CIF_verification.xlsx")

In [100]:
df.to_excel("checkpoint_CIF_verification_labels.xlsx")

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 88 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Perovskite                            1089 non-null   object 
 1   Hill formula                          730 non-null    object 
 2   Interlayer space composition          5 non-null      object 
 3   Bandgap, eV                           1054 non-null   float64
 4   DOI                                   1084 non-null   object 
 5   Materials Project ID                  1058 non-null   object 
 6   COD_ID                                935 non-null    object 
 7   Springer_ID                           290 non-null    object 
 8   MP_CIF_modifier                       499 non-null    object 
 9   COD_CIF_modifier                      197 non-null    object 
 10  Springer_CIF_modifier                 121 non-null    object 
 11  Materials Project

#CIF intercalation modifier


In [102]:
!pip install ase
!pip install rdkit

In [103]:

import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from scipy.spatial.transform import Rotation as R
from rdkit.Chem import rdDetermineBonds

In [104]:
#smiles = "CCCCCCCCN"
#mol = Chem.MolFromSmiles(smiles)
#mol = Chem.MolFromXYZFile('octhylamine_text.xyz')
#conn_mol = Chem.Mol(mol)
#conn_mol = mol
#rdDetermineBonds.DetermineBondOrders(conn_mol,charge=0)
#rdDetermineBonds.DetermineBonds(conn_mol)
#mol = Chem.AddHs(conn_mol)

#AllChem.EmbedMolecule(mol, randomSeed=42)
#AllChem.UFFOptimizeMolecule(mol)

#conf = conn_mol.GetConformer()
#coords = np.array([conf.GetAtomPosition(i) for i in range(conn_mol.GetNumAtoms())])
#print(coords)